### Attach Google Drive for Data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

## Install required libraries

In [ ]:
!pip install super-gradients
!pip install -q supervision

## Library import

In [ ]:
from super_gradients.training import Trainer, models
from super_gradients.training.dataloaders.dataloaders import coco_detection_yolo_format_train, coco_detection_yolo_format_val
from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import DetectionMetrics_050, DetectionMetrics_050_095
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback

### Import Dataset

In [ ]:
# Dataset
directory = '/content/drive/MyDrive/Dissertation/YOLONAS-DEEPSORT/Data'
obj_class = ['Cell']

In [ ]:
dataset_params = {
    'data_dir': directory,
    'train_images_dir':'train/images',
    'train_labels_dir':'train/labels',
    'val_images_dir':'valid/images',
    'val_labels_dir':'valid/labels',
    'test_labels_dir':'test/labels',
    'test_labels_dir':'test/labels',
    'classes': obj_class
}

### Parameters

In [ ]:
batch_size = 8
epochs = 20

## Initialising YOLONAS Instance

In [ ]:
# Instance of training and validation dataset
train_data = coco_detection_yolo_format_train(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['train_images_dir'],
        'labels_dir': dataset_params['train_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': batch_size,
        'num_workers': 2
    }
)

val_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['val_images_dir'],
        'labels_dir': dataset_params['val_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': batch_size,
        'num_workers': 2
    }
)

In [ ]:
train_data.dataset.transforms

### Training Parameters

In [ ]:

train_params = {
    'silent_mode': False,
    "average_best_models":True,
    "warmup_mode": "linear_epoch_step",
    "warmup_initial_lr": 1e-6,
    "lr_warmup_epochs": 3,
    "initial_lr": 5e-4,
    "lr_mode": "cosine",
    "cosine_final_lr_ratio": 0.1,
    "optimizer": "Adam",
    "optimizer_params": {"weight_decay": 0.0001},
    "zero_weight_decay_on_bias_and_bn": True,
    "ema": True,
    "ema_params": {"decay": 0.9, "decay_type": "threshold"},
    "max_epochs": epochs,
    "mixed_precision": True,
    "loss": PPYoloELoss(
        use_static_assigner=False,
        num_classes=len(dataset_params['classes']),
        reg_max=16
    ),
    "valid_metrics_list": [
        DetectionMetrics_050(
            score_thres=0.1,
            top_k_predictions=300,
            num_cls=len(dataset_params['classes']),
            normalize_targets=True,
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01,
                nms_top_k=1000,
                max_predictions=300,
                nms_threshold=0.7
            )
        ),
        DetectionMetrics_050_095(
            score_thres=0.1,
            top_k_predictions=300,
            num_cls=len(dataset_params['classes']),
            normalize_targets=True,
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01,
                nms_top_k=1000,
                max_predictions=300,
                nms_threshold=0.7
            )
        )
    ],
    "metric_to_watch": 'mAP@0.50'
}

### Model Training

In [ ]:
# Training model YOLONAS-S, YOLONAS-M, YOLONAS-L
models_to_train = [
    'yolo_nas_s',
    'yolo_nas_m',
    'yolo_nas_l'
]

# Directory of checkpoint
checkpoint_dir = '/content/drive/MyDrive/Dissertation/YOLONAS-DEEPSORT/checkpoints/{}'

for model_to_train in models_to_train:
  model = models.get(
    model_to_train,
    num_classes=len(dataset_params['classes']),
    pretrained_weights="coco"
    )

  trainer = Trainer(experiment_name='ISC-Detection', ckpt_root_dir=checkpoint_dir.format(model_to_train))

  trainer.train(
        model=model,
        training_params=train_params,
        train_loader=train_data,
        valid_loader=val_data
    )

In [ ]:
# View results
%load_ext tensorboard
%tensorboard --logdir {'/content/drive/MyDrive/Dissertation/YOLONAS-DEEPSORT/checkpoints/'} --port=8008

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
# !zip -r yolo_nas.zip {checkpoint_dir}/{'ISC-Detection'}